# Loading In and Preprocessing the Data

**Importing Libraries**

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from ucimlrepo import fetch_ucirepo 

**Loading In Data (Using the UCI Machine Learning Repository to Load in Data)**

In [92]:
ratings = fetch_ucirepo(id=485) 
  

data = pd.DataFrame(ratings.data.features)

display(data.head())
display(data.tail())
data.shape

,churches,resorts,beaches,parks,theatres,museums,malls,zoos,restaurants,pubs/bars,...,art galleries,dance clubs,swimming pools,gyms,bakeries,beauty & spas,cafes,view points,monuments,gardens
0,0.0,0.0,3.63,3.65,5.0,2.92,5.0,2.35,2.33,2.64,...,1.74,0.59,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.63,3.65,5.0,2.92,5.0,2.64,2.33,2.65,...,1.74,0.59,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,3.63,3.63,5.0,2.92,5.0,2.64,2.33,2.64,...,1.74,0.59,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0
3,0.0,0.5,3.63,3.63,5.0,2.92,5.0,2.35,2.33,2.64,...,1.74,0.59,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,3.63,3.63,5.0,2.92,5.0,2.64,2.33,2.64,...,1.74,0.59,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0


,churches,resorts,beaches,parks,theatres,museums,malls,zoos,restaurants,pubs/bars,...,art galleries,dance clubs,swimming pools,gyms,bakeries,beauty & spas,cafes,view points,monuments,gardens
5451,0.91,5.00,4.00,2.79,2.77,2.57,2.43,1.09,1.77,1.04,...,5.00,0.66,0.65,0.66,0.69,5.00,1.05,5.0,5.0,1.56
5452,0.93,5.00,4.02,2.79,2.78,2.57,1.77,1.07,1.76,1.02,...,0.89,0.65,0.64,0.65,1.59,1.62,1.06,5.0,5.0,1.09
5453,0.94,5.00,4.03,2.80,2.78,2.57,1.75,1.05,1.75,1.00,...,0.87,0.65,0.63,0.64,0.74,5.00,1.07,5.0,5.0,1.11
5454,0.95,4.05,4.05,2.81,2.79,2.44,1.76,1.03,1.74,0.98,...,5.00,0.64,0.63,0.64,0.75,5.00,1.08,5.0,5.0,1.12
5455,0.95,4.07,5.00,2.82,2.80,2.57,2.42,1.02,1.74,0.96,...,0.85,0.64,0.62,0.63,0.78,5.00,1.08,5.0,5.0,1.17


(5456, 24)

**Looking at Data Frame Information**

In [93]:
data.dtypes

churches                 float64
resorts                  float64
beaches                  float64
parks                    float64
theatres                 float64
museums                  float64
malls                    float64
zoos                     float64
restaurants              float64
pubs/bars                float64
local services            object
burger/pizza shops       float64
hotels/other lodgings    float64
juice bars               float64
art galleries            float64
dance clubs              float64
swimming pools           float64
gyms                     float64
bakeries                 float64
beauty & spas            float64
cafes                    float64
view points              float64
monuments                float64
gardens                  float64
dtype: object

All values in the dataset should be floating point numbers between 0 and 5, local services is categroized as a object datatype which it could have string values, so for the clustering algorithms to work we need to convert the values into float number values.

In [94]:
#data['local services'] = data['local services'].astype(float)

#Cant use astype method to convert the local services column into float values because it raises a value error, it appears there are string values in this column

#To get around this conversion error I'm going to use the pd.to_numeric method to replace any non-number values with an NaN value
data['local services'] = pd.to_numeric(data['local services'], errors='coerce')

**Checking For Missing/NaN values in Dataset**

In [95]:
print('Amount of Missing Values For Each Feature')
data.isnull().sum() #Appears to be some missing/NaN values so we should get rid of those rows since the dataset is pretty big anyways

Amount of Missing Values For Each Feature


churches                 0
resorts                  0
beaches                  0
parks                    0
theatres                 0
museums                  0
malls                    0
zoos                     0
restaurants              0
pubs/bars                0
local services           1
burger/pizza shops       1
hotels/other lodgings    0
juice bars               0
art galleries            0
dance clubs              0
swimming pools           0
gyms                     0
bakeries                 0
beauty & spas            0
cafes                    0
view points              0
monuments                0
gardens                  1
dtype: int64

In [96]:
data = data.dropna() #Removing any rows with a missing or NaN value
data.shape #Two rows appear to be dropped

(5454, 24)